In [1]:
from torch.utils.data import DataLoader
import torch

from src.data.data_loader import StreetHazardsDataset
from src.models.model import OSNet
from src.utils.loss import CrossEntropyLoss
CUDA_LAUNCH_BLOCKING=1

In [4]:

from src.models.modeling import deeplabv3_resnet50, deeplabv3plus_resnet50, deeplabv3_resnet34

train_loader = StreetHazardsDataset("train")
test_loader = StreetHazardsDataset("test")
val_loader = StreetHazardsDataset("val")

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Device: %s" % device)

model = deeplabv3_resnet50(num_classes=12)
model.to(device)
train_dataloader = DataLoader(train_loader, batch_size=3, shuffle=True)


Device: cpu


In [5]:


from src.utils.loops import train_loop

loss_fn = CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer,device,batch_size=3)
    #test_loop(test_dataloader, model, loss_fn)
print("Done!") 

Epoch 1
-------------------------------


0it [00:00, ?it/s]C:\Users\TechRufy\PycharmProjects\ML4CV\.venv\Lib\site-packages\torch\nn\_reduction.py:51: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))
0it [00:56, ?it/s]


IndexError: Target 13 is out of bounds.